# Matemática Computacional II
- Prof. Felipe C. Minuzzi
- felipe.minuzzi@ufsm.br

In [15]:
import numpy as np
import matplotlib.pyplot as plt

## Busca virtual pela internet - uma aplicação de matrizes e sistemas

Imagine que decidimos olhar no Google quem foram as personalidades nascidas em 22 de maio. Em 0.40 segundos, a plataforma encontra 2.020.000.000 resultados (não se engane nos zeros - sim, são 2 BILHÕES de resultados). Clicando na primeira página, da Wikipedia, descobrimos que nesse dia, mas em 1858, nasceu Sir Artur Conan Doyle na Escócia. Nossa mente rapidamente associa ao autor à sua princial criação, Sherlock Holmes e, digitando na barra de busca, obtemos 107.000.000 resultados em apenas 0.29 segundos. Entre os links que aparecem, podemos escolher um que cita o autor Robert Downey Jr, e daí seguimos para os Vingadores, Guerra Infinita... e assim por diante.

O divertido aqui é que o Google nos levou por um caminho que praticamente antecipa nossos pensamentos. Quando digitamos uma ou mais palavras-chaves em um buscador, como o Google, em um espaço de tempo muito curto, em geral menor que um segundo, conseguimos uma lista de endereços de páginas, chamadas URLs (<em> Uniform Resource Locator</em> ), relacionadas às palavras-chaves que fornecemos. É mais impressionante ainda pensarmos que a ordem do resultado da busca está associado a relevância da página e seu contexto nas palavras-chaves.


<img src="./figs/google1.png" width="750">


A pergunta que podemos fazer é: 

**Como o Google responde de maneira tão rápida nossas buscas? E mais, como organiza a lista de resultado baseado na sua relevância? Por qual critério as páginas são listadas?**

### O algoritmo <em> PageRank </em> 

O método <em> PageRank </em> de organizar as páginas da web é a estratégia utilizada pelo Google para poder ordenar e classificar os resultadas das buscas que são realizadas na sua plataforma. Consiste em um número que calcula a relevância da página com relação as outras. Quanto maior o <em> PageRank </em> de uma página, mais relevante ela é $^1$.

Toda página analisada pelo Google $^2$ possui o seu próprio <em> PageRank </em>, calculado por um algoritmo sofisticado independentemente das palaras utilizadas na busca, e continuamente atualizado pela evolução da rede. 

De modo a gerar o ranking de todas páginas da web, o <em> PageRank </em> (a partir daqui, vamos citá-lo apenas como PR) simula o comportamente de alguém que está navegando pela rede de modo randômico e incessantemente, fazendo buscas e seguindo uma trilha de links de uma página a outra. O PR é um indíce numérico que leva em consideração apenas a estrutura da web (isto é, como as páginas estão relacionadas entre si), e não acessa o conteúdo das páginas, não analisa a veracidade das informações ou julga o conteúdo.

A partir dessas informações, vamos estudar como construir um modelo matemático e depois usar uma estratégia numérica para calcular o <em> PageRank </em> de uma rede relativamente simples. Com isso, poderemos listar as páginas a partir do nosso resultado, do maior para o menor indíce.


<font size="2">1 O que vamos falar aqui é apenas a versão pública do algoritmo. Provavelmente essa versão já é obsoleta, pois a atual é guardada a sete chaves.</font> 

<font size="2">2 Em 2024, número total de páginas rankeadas pelo Google é de 60 bilhões.</font>

### Utilizando grafos para modelar uma rede

Considere uma rede (muito) pequena dada pela figura abaixo:

<img src="./figs/google2.png" width="600">

Essa rede será modelada por um grafo, ou melhor, um grafo direcionado, que nada mais é que uma coleção de elementos (chamados nós) e conexões (chamadas arestas) orientadas de um nó para o outro. No nosso caso, cada página será representada por um nó e cada aresta é um link entre duas páginas, conforme a figura abaixo:


<img src="./figs/google3.png" width="450">

A informação gerada pelo gráfico acima, ou sejam o número $N$ de nós e arestas pode ser armazenado em uma matriz, chamada de **matriz de adjacência de um grafo**:

A matriz de adjacência $A$ de um grafo possui $N$ linhas e colunas e seus elementos $a_{ij}$ são dados por

$$a_{ij} = \left\{ \begin{array}{ll}
            1, \text{se existe um link do nó j para o nó i;} \\
            0, \text{o contrário.}
	       \end{array} \right.$$

Para o nosso exemplo da rede acima, a matriz de adjacência $A$ é dada por:

$$A = \left[\begin{array}{rrrrr}
0 & 0 & 1 & 0 \\ 
1 & 0 & 1 & 0 \\ 
1 & 0 & 0 & 0 \\ 
0 & 1 & 1 & 0 
\end{array}\right]$$

Observe que a primeira coluna armazena a informação dos links originarios do primeiro nó e apontando para os outros nós. Olhando a matriz, podemos inferir que dois links saem do nó 1: um para o nó 2 (e portanto $a_{21} = 1$) e um para o nó 3 ($a_{31} = 1$).

A segunda coluna de $A$ possui apenas um elemento não nulo, o $a_{42}$. De fato, existe apenas um link saindo do nó 2 para o nó 4, como podemos confirmar pela figura acima. A terceira coluna possui todos elementos iguais a 1 exceto o da diagonal principal ($a_{33}$), ou seja, existe links saindo do nó 3 para todos outros nós, exceto ele mesmo. A última coluna, a quarta, possui todos elementos zerados: nenhum link sai do nó 4 para outros nós.


In [16]:
#vamos criar a matriz A:

A = np.array([
    [0,0,1,0],
    [1,0,1,0],
    [1,0,0,0],
    [0,1,1,0]
])
print(f'A matriz de adjacência do grafo é: {A}')

A matriz de adjacência do grafo é: [[0 0 1 0]
 [1 0 1 0]
 [1 0 0 0]
 [0 1 1 0]]


### Da matriz de adjacência para a matriz Google

Vamos agora indicar por $L_j$ o número de conexões saindo do nó $j$, para todo $j = 1, \ldots, N$. Definimos agora uma nova matriz $H$ de ordem $N$ cujos elementos $h_{ij}$ são dados por:

$$h_{ij} = \left\{ \begin{array}{ll}
            \dfrac{1}{L_j}, \ \ \text{se} \ \ L_{ij}  \neq 0 \ \ \text{e existe um link do nó j para o nó i;} \\
            0, \text{o contrário.}
	       \end{array} \right.$$

Para o nosso exemplo da rede acima, temos que $L_1 = 2$. $L_2 = 1$, $L_3 = 3$ e $L_4 = 0$. Assim, montamos a matriz H:

$$H = \left[\begin{array}{rrrrr}
0           & 0 & \frac{1}{3} & 0 \\ 
\frac{1}{2} & 0 & \frac{1}{3} & 0 \\ 
\frac{1}{2} & 0 & 0           & 0 \\ 
0           & 1 & \frac{1}{3} & 0 
\end{array}\right]$$

Note agora que os elementos de $H$ são números entre $0$ e $1$ e a soma de todos elementos de uma **coluna** é sempre um, exceto quando não existe link saindo do nó (como o caso da coluna 4 da matriz acima).

In [17]:
#vamos criar a matriz H:

H = np.array([
    [0,  0,1/3,0],
    [1/2,0,1/3,0],
    [1/2,0,0,  0],
    [0,  1,1/3,0]
])
print(f'A matriz H é: {H.round(3)}')

A matriz H é: [[0.    0.    0.333 0.   ]
 [0.5   0.    0.333 0.   ]
 [0.5   0.    0.    0.   ]
 [0.    1.    0.333 0.   ]]


**O que os elementos da matriz H parecem?**

A matriz H na verdade é uma matriz de possibilidades, ou uma matriz de probabilidades, onde cada elemento $h_{ij}$ representa a probabilidade de algum usuário que está na página $j$ da rede clicar no link que o leva da página $j$ para a página $i$.

É importante salientar que tanto a matriz $A$ como a matrz $H$ possuem informações apenas das conexões entre páginas, e nada relativo ao conteúdo dessas.

**Mas o que acontece com o usuario que chega na página do nó 4?**

Note que a matriz $H$ não fornece exatamente toda informação que precisamos. Geralmente, um usuário que chega em um página que não possui nenhum link, simplesmente começa uma nova busca ou digita outro site para navegar. Precisamos adicionar essa informação na nossa matriz, ou seja, a ação do usuário sair de uma página como a do nó 4 e ir para outra. Os valores nulos da quarta coluna da matriz $H$ devem ser substituídos por algum valor mais concreto, ou melhor, uma probabilidade do usuário sair da página do nó 4 e ir para outra por meio de uma nova busca, e não pelo link.

Como, <em> a priori </em> , não existe preferência por págna, assumiremos que a probabilidade para acessar uma nova página é igual para todas, isto é, vale $1/N$. Portanto, substituímos a matriz $H$ pela matriz $S$ tal que:

 - se $j$ é um nó nulo (sem links para outros nós), todos elementos da coluna $j$ são definidos como $1/N$;
 - do contrário, definimos $s_{ij} = h_{ij}$, ou seja, mantemos a informação da matriz $H$.

 Precisamente, temos:

$$s_{ij} = \left\{ \begin{array}{ll}
            \dfrac{1}{N}, \ \ \text{se} \ \ L_{ij}  = 0; \\
            \dfrac{1}{L_j}, \ \ \text{se} \ \ L_{ij}  \neq 0 \ \ \text{e existe um link do nó j para o nó i;} \\
            0,, \ \ \text{se} \ \ L_{ij}  \neq 0 \ \ \text{e não existe um link do nó j para o nó i.} \\
	       \end{array} \right.$$

Para a nossa rede, temos:

$$H = \left[\begin{array}{rrrrr}
0           & 0 & \frac{1}{3} & \frac{1}{4} \\ 
\frac{1}{2} & 0 & \frac{1}{3} & \frac{1}{4} \\ 
\frac{1}{2} & 0 & 0           & \frac{1}{4} \\ 
0           & 1 & \frac{1}{3} & \frac{1}{4} 
\end{array}\right]$$

In [18]:
#vamos criar a matriz S:

S = np.array([
    [0,  0,1/3,1/4],
    [1/2,0,1/3,1/4],
    [1/2,0,0,  1/4],
    [0,  1,1/3,1/4]
])
print(f'A matriz S é: {S.round(3)}')

A matriz S é: [[0.    0.    0.333 0.25 ]
 [0.5   0.    0.333 0.25 ]
 [0.5   0.    0.    0.25 ]
 [0.    1.    0.333 0.25 ]]


Agora, a soma dos elementos de qualquer coluna é sempre um:

$$\sum_{i = 1}^N a_{ij} = 1 \ \ \ \ \forall \ j = 1, \ldots, N$$ 

Para criar a matriz Google, utilizada pela plataforma, precisamos considerar que os usuários não apenas seguem links: mesmo tendo um, alguém pode simplesmente mudar de páginas digitando um novo endereço ou começando uma nova busca. Para simular essa mudança de caminho, introduzimos um novo parâmetro $\alpha$, de modo que $\alpha \in \mathbb{R}$ e $0 \leq \alpha \leq 1$, e definimos a matriz Google $G$, cujos elementos são dados por:

$$g_{ij} = \alpha s_{ij} + \dfrac{1 - \alpha}{N}$$

O elemento $g_{ij}$ expressa a probabilidade de um usuário, visitando a página $j$, trocar para a página $i$. Observe que cada elemento $g_{ij}$ representa um número real entre $0$ e $1$, e a soma dos elementos de cada coluna de $G$ é sempre $1$. Matematicamente:


$$\begin{array}{ll}
            0 \leq g_{ij} \leq 1 & \forall i,j = 1, \ldots, N; \\
            g_{1j} + g_{2j} + g_{3j} + \ldots + g_{Nj} = 1 & \forall j = 1, \ldots, N
	       \end{array} 
           $$

O que acontece se $\alpha$ for igual a zero? Temos que $g_{ij} = \dfrac{1}{N}$ para todo par $i$ e $j$ de nós. Nesse caso, a matriz $G$ modela um usuário que navega pelas páginas pulando de uma para a outra com a mesma probabilidade. Quando $\alpha$ é diferente de zero, quanto maior ele for, maior é a probabilidade de um usuário seguir o caminho pelos links das páginas, e não começar uma nova busca ou digitar um novo site. 

Abaixo, construimos a matriz $G$ para nossa rede de 4 páginas usando $\alpha = 0.85$.

In [22]:
#definimos parametros iniciais
N     = 4
alpha = 0.85

# primeiro vamos criar uma matriz nula de ordem 4
G = np.zeros((N,N))

#agora calculamos cada gij baseado na fórmula acima
for i in range(N):
    for j in range(N):

        G[i,j] = alpha*S[i,j] + (1 - alpha)/N

print(G)


[[0.0375     0.0375     0.32083333 0.25      ]
 [0.4625     0.0375     0.32083333 0.25      ]
 [0.4625     0.0375     0.0375     0.25      ]
 [0.0375     0.8875     0.32083333 0.25      ]]


## O modelo matemático

Até agora, não calculamos efetivamente o valor do <em> PageRank </em> das páginas. Criamos uma matriz que nos ajuda nesse sentido, baseado em probabilidades. Seja, portanto, $p_i$ o <em> PageRank </em> de uma página $i$ qualquer. Esse número representará uma probabilidade, logo, 

$$ 0 \leq p_i \leq 1 \ \ \ \ \forall i = 1, \ldots, N.$$

Vamos supor também que

$$p_i + p_2 + p_3 + \ldots + p_N$$

Vamos computar o valor de $p_i$ utilizando as informações da matriz Google $G$. Começaremos com a página do nó 2 da nossa rede dada lá no início do notebook. Um usuário qualquer pode chegar à página 2 a partir de um link de uma outra página ou começando uma busca que inicia na página 2. Vamos considerar a primeira opção, onde nosso usuário chegou a página 2 a partir de um link na página 1. Na matriz Google, a probabilidade dessa situação é dada por $g_{21}$ (lembre que esse valor é a probabilidade de um usuário que está na página 1 seguir para a página 2). Agora, o fato do usuário estar na página um não é um evento certo, isto é, não podemos assumir isso com nenhuma certeza. A probabilidade do usuário estar na página 1 é exatamente o <em> PageRank </em> da página 1, $p_1$.

**Resumindo:** a probabilidade de um usuário chegar à página 2 a partir da página um é **CONDICIONAL**, isto é, depende da condição dele já estar na página 1. A isso damos o nome de probabilidade condicional, e essa é calculada pelo produto $g_{21}p_1$. 

De maneira análoga, $g_{22}p_2$ expressa a probabilidade de se manter na página 2; $g_{23}p_3$ a probabilidade de chegar à página 2 a partir da página 3, condicionando o fato de já estar na página 3 e por fim, $g_{24}p_4$ a probabilidade de chegar à página 2 a partir da página 4, condicionando o fato de já estar na página 4. 

Portanto, a **probabilidade total de visitar a página 2** é dada por:

$$p_2 = g_{21}p_1 + g_{22}p_2 + g_{23}p_3 + g_{24}p_4 $$

Essa fórmula não aparece 'do nada'. Ela é um resultado da lei de probabilidade total, cuja hipótese fundamental de aplicação é assumir que o usuário não pode visitar duas páginas ao mesmo tempo (isto é, ele não fica abrindo várias abas no navegador!!). 

Seguindo o mesmo argumento para obter o <em> PageRank </em> das outras páginas, temos

$$\left\{ \begin{array}{l}
            p_1 = g_{11}p_1 + g_{12}p_2 + g_{13}p_3 + g_{14}p_4; \\
            p_2 = g_{21}p_1 + g_{22}p_2 + g_{23}p_3 + g_{24}p_4; \\
            p_3 = g_{31}p_1 + g_{32}p_2 + g_{33}p_3 + g_{34}p_4; \\
            p_4 = g_{41}p_1 + g_{42}p_2 + g_{43}p_3 + g_{44}p_4.
	       \end{array} \right.$$

Cada <PageRank> depende do índice das outras páginas. Chegamos, assim, em um sistema $4 \times 4$ que nos permite calcular os índices das 4 páginas a partir da rede que consideramos. Generalizando para uma rede com dimensão $N$, isto é, com $N$ páginas, temos:

$$\left\{ \begin{array}{l}
            p_1 = g_{11}p_1 + g_{12}p_2 + g_{13}p_3 + \ldots + g_{1N}p_N; \\
            p_2 = g_{21}p_1 + g_{22}p_2 + g_{23}p_3 + \ldots + g_{2N}p_N \\
            p_3 = g_{31}p_1 + g_{32}p_2 + g_{33}p_3 + \ldots + g_{3N}p_N \\
            \vdots \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \vdots \ \ \ \ \ \ \ \ \ \ \ \ \  \ \ \ \ \ \ \ \vdots \ \ \ \ \ \ \  \ \ \ \ \ \ \ \ \ \ \ \ \ \vdots\\
            p_N = g_{N1}p_1 + g_{N2}p_2 + g_{N3}p_3 + \ldots + g_{NN}p_N
	       \end{array} \right.$$

Usando a matriz Google $G$ generalizada e colocando todos <em> PageRanks </em> em um vetor ${\bm p} = [p_1, p_2, p_3, \ldots, p_N] $, o sistema acima pode ser escrito como

$$ {\bm p} = G{\bm p} $$

Enfim, chegamos em um modelo matemático para calcular o <em> PageRanks </em> das páginas da rede. O sistema obtido acima nos permite calcular o índice para as $N$ páginas de uma rede. As condições

$$\begin{array}{ll}
            0 \leq g_{ij} \leq 1 & \forall i,j = 1, \ldots, N; \\
            g_{1j} + g_{2j} + g_{3j} + \ldots + g_{Nj} = 1 & \forall j = 1, \ldots, N
	       \end{array} 
           $$

garantem que o sistema ${\bm p} = G{\bm p}$ possui uma solução única, ou seja, existe apenas um índice <em> PageRank </em> para cada página da rede.

### Como resolver o problema?

O sistema ${\bm p} = G{\bm p}$ não é usual, tendo em vista que a solução ${\bm p}$ aparece em ambos os lados da equação. Dizemos que o sistema é **implícito**, o que significa que ${\bm p}$ é conhecido apenas implicitamente em termos de si mesmo. 

Uma alternativa para resolver o sistema é usar uma estratégia parecida com a dos métodos de Jacobi e Gauss-Seidel, ou seja, criar um processo iterativo a partir de ${\bm p} = G{\bm p}$. Assim, podemos iniciar nosso processo com o vetor solução ${\bm p}$ da primeira iteração dado por 

$$ {\bm p}^{(0)}  = \left[\begin{array}{c}
\frac{1}{N} \\ 
\frac{1}{N} \\ 
\frac{1}{N} \\ 
\vdots \\
\frac{1}{N} 
\end{array}\right]$$

o que significa que, inicialmente, a probabilidade do usuário acessar qualquer página é sempre a mesma. Iteramos a partir da equação ${\bm p} = G{\bm p}$ para obter uma segunda estimativa:

$${\bm p}^{(1)} = G{\bm p}^{(0)}$$

e com isso, podemos seguir para uma terceira estimativa:

$${\bm p}^{(2)} = G{\bm p}^{(1)}$$

e assim por diante. Logo, o processo numérico iterativo para solução do sistema é, dado uma estimativa inicial ${\bm p}^{(0)}$, para $k = 1, 2, \ldots $, calcular 

$${\bm p}^{(k)} = G{\bm p}^{(k - 1)}$$

Como vimos, o critério de parada será a partir do erro relativo entre cada uma das iterações, até atingir uma tolerância $\epsilon$ desejada, isto é, 

$$ e_{N}^{k} = \dfrac{|{\bm p}^{(k)} - {\bm p}^{(k-1)}|}{|{\bm p}^{(k)}|} < \epsilon$$

Vamos agora calcular a solução para os <em> PageRanks </em> da nossa rede de 4 páginas.



In [31]:
#primeiro, definimos o vetor inicial p
p = [1/N for n in range(N)]

#tolerancia
tol  = 1e-3
pant = p.copy() 

#criamos a variavel erro para controlar a iteração
erro = 1
it   = 0

while erro > tol:

    #colocamos a formula usando .dot(), pois queremos produto matriz por vetor
    p = G.dot(p)

    #calculamos o erro
    erro = abs(max(p - pant))/max(abs(p))

    #atualizamos a variavel
    pant = p

    #contagem de iterações:
    it += 1
    print(f'Iteração {it} ----- Solução: {p} ----- Erro: {erro}')

print(f'Vetor solução final: {p}')
    

Iteração 1 ----- Solução: [0.16145833 0.26770833 0.196875   0.37395833] ----- Erro: 0.33147632311977704
Iteração 2 ----- Solução: [0.1727474  0.24136719 0.18558594 0.40029948] ----- Erro: 0.06580359756692587
Iteração 3 ----- Solução: [0.17514632 0.24856396 0.19598128 0.38030843] ----- Erro: 0.027333985273568832
Iteração 4 ----- Solução: [0.17384357 0.24828076 0.19275273 0.38512294] ----- Erro: 0.012501230768870556
Iteração 5 ----- Solução: [0.1739519  0.24783542 0.19322214 0.38499054] ----- Erro: 0.001219289171885887
Iteração 6 ----- Solução: [0.17405676 0.24798632 0.19324005 0.38471687] ----- Erro: 0.00039224903956675374
Vetor solução final: [0.17405676 0.24798632 0.19324005 0.38471687]


O algoritmo parou em 6 iteraçõs, obtendo o vetor solução final. Com ele, temos o índice <em>PageRank</em> de cada uma das páginas, na ordem 1,2,3,4:

In [32]:
for i in range(N):
    print(f'p{i+1} ----- {p[i]}')

p1 ----- 0.17405676419671373
p2 ----- 0.24798632090367206
p3 ----- 0.19324004703341588
p4 ----- 0.38471686786619813


Como conclusão, temos que a página 4 tem o maior <em>PageRank</em>, seguido da página 2, depois a página 3 e por fim a página 1. Essa seria a ordem em que apareceriam as páginas em uma busca pelo Google!

#### Exercícios

Abaixo, dois exercícios para treinar o que aprendemos nesse notebook!

**Exercício 1:**

Encontre os índices <em>PageRanks</em> da rede abaixo:

<img src="./figs/google4.png" width="750">



**Exercício 2:**

Encontre os índices <em>PageRanks</em> para uma rede cujo grafo é dado abaixo:

<img src="./figs/google5.png" width="750">